# Creating an Einstein Toolkit thorn

### Now that we have coded of the Weyl scalars and associated invariants, we need to create a thorn to incorporate the generated C code into the Toolkit to calculate the scalars.

To create an Einstein Toolkit thorn to numerically compare the two codes, we will need a few more files, as required by the Einstein Toolkit: the scheduling, parameter, and interface $\text{.ccl files}$ (our specific thorn does not require the optional $\text{configuration.ccl}$ file). 

We will start with the file $\text{interface.ccl}$. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the scalars $\text{psi4r}$
and $\text{psi4i}$ to be visible to other thorns by using the keyword "public". 

In [35]:
%%writefile WeylScal4NRPy/interface.ccl

implements: WeylScal4NRPy

inherits:   admbase Boundary Grid methodoflines

public:
CCTK_REAL NRPyPsi4r_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi4r
} "Psi4r_group"

public:
CCTK_REAL NRPyPsi4i_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=-1'
{
  psi4i
} "Psi4i_group"

public:
CCTK_REAL NRPyPsi3r_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi3r
} "Psi3r_group"

public:
CCTK_REAL NRPyPsi3i_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=-1'
{
  psi3i
} "Psi3i_group"

public:
CCTK_REAL NRPyPsi2r_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi2r
} "Psi2r_group"

public:
CCTK_REAL NRPyPsi2i_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=-1'
{
  psi2i
} "Psi2i_group"

public:
CCTK_REAL NRPyPsi1r_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi1r
} "Psi1r_group"

public:
CCTK_REAL NRPyPsi1i_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=-1'
{
  psi1i
} "Psi1i_group"

public:
CCTK_REAL NRPyPsi0r_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  psi0r
} "Psi0r_group"

public:
CCTK_REAL NRPyPsi0i_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=-1'
{
  psi0i
} "Psi0i_group"

public:
CCTK_REAL NRPyCurvIr_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvIr
} "CurvIr_group"

public:
CCTK_REAL NRPyCurvIi_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvIi
} "CurvIi_group"

public:
CCTK_REAL NRPyCurvJr_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJr
} "CurvJr_group"

public:
CCTK_REAL NRPyCurvJi_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJi
} "CurvJi_group"

public:
CCTK_REAL NRPyCurvJ1_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJ1
} "CurvJ1_group"

public:
CCTK_REAL NRPyCurvJ2_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJ2
} "CurvJ2_group"

public:
CCTK_REAL NRPyCurvJ3_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJ3
} "CurvJ3_group"

public:
CCTK_REAL NRPyCurvJ4_group type=GF timelevels=3 tags='tensortypealias="Scalar" tensorweight=0 tensorparity=1'
{
  NRPycurvJ4
} "CurvJ4_group"



Overwriting WeylScal4NRPy/interface.ccl


We will now write the file $\text{param.ccl}$. This file allows the listed parameters to be set at runtime. We also give allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). Here, the first two parameters tell the Method of Lines thorn that none of our variables need to be evolved in time. The last two determine how often we will calculate $\psi_4$ and how many timesteps to skip at the beginning; currently, they are set to calculate $\psi_4$ at every timestep, starting from the first in the simulation. 

In [36]:
%%writefile WeylScal4NRPy/param.ccl

restricted:
CCTK_INT WeylScal4NRPy_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of evolved variables used by this thorn"
} 0

restricted:
CCTK_INT WeylScal4NRPy_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_INT WeylScal4NRPy_calc_every "WeylScal4_psi4_calc_Nth_calc_every" STEERABLE=ALWAYS
{
  *:* :: ""
} 1

restricted:
CCTK_INT WeylScal4NRPy_calc_offset "WeylScal4_psi4_calc_4th_calc_offset" STEERABLE=ALWAYS
{
  *:* :: ""
} 0

private:
CCTK_BOOLEAN output_all_psis "Whether to output all Weyl scalars or just psi4" 
{
} "true"

private:
CCTK_BOOLEAN output_invars "Whether or not to output the invariant scalars" 
{
} "true"

Overwriting WeylScal4NRPy/param.ccl


Finally, we will write the file $\text{schedule.ccl}$; its official documentation is found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). This file dictates when the various parts of the thorn will be run. We first assign storage for both the real and imaginary components of $\psi_4$, and then specify that we want our code run in  the $\text{MoL_PseudoEvolution}$ schedule group, after the ADM variables are set. At this step, we declare that we will be writing code in C. We also specify the gridfunctions that we wish to read in from memory--in our case, we need all the components of $K_{ij}$ and $g_{ij}$ (which we have been calling $\gamma_{ij}$) in addition to the coordinate values.

In [37]:
%%writefile WeylScal4NRPy/schedule.ccl

STORAGE: NRPyPsi4r_group[timelevels]
STORAGE: NRPyPsi4i_group[timelevels]
STORAGE: NRPyPsi3r_group[timelevels]
STORAGE: NRPyPsi3i_group[timelevels]
STORAGE: NRPyPsi2r_group[timelevels]
STORAGE: NRPyPsi2i_group[timelevels]
STORAGE: NRPyPsi1r_group[timelevels]
STORAGE: NRPyPsi1i_group[timelevels]
STORAGE: NRPyPsi0r_group[timelevels]
STORAGE: NRPyPsi0i_group[timelevels]
STORAGE: NRPyCurvIr_group[timelevels]
STORAGE: NRPyCurvIi_group[timelevels]
STORAGE: NRPyCurvJr_group[timelevels]
STORAGE: NRPyCurvJi_group[timelevels]
STORAGE: NRPyCurvJ1_group[timelevels]
STORAGE: NRPyCurvJ2_group[timelevels]
STORAGE: NRPyCurvJ3_group[timelevels]
STORAGE: NRPyCurvJ4_group[timelevels]

schedule group WeylScal4NRPy_group in MoL_PseudoEvolution after ADMBase_SetADMVars
{
} "Schedule WeylScal4NRPy group"

schedule weylscal4_mainfunction in WeylScal4NRPy_group
{
  LANG: C
  READS: admbase::kxx(Everywhere)
  READS: admbase::kxy(Everywhere)
  READS: admbase::kxz(Everywhere)
  READS: admbase::kyy(Everywhere)
  READS: admbase::kyz(Everywhere)
  READS: admbase::kzz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::z(Everywhere)
  WRITES: WeylScal4::psi4i(Interior)
  WRITES: WeylScal4::psi4r(Interior)
  WRITES: WeylScal4::psi3i(Interior)
  WRITES: WeylScal4::psi3r(Interior)
  WRITES: WeylScal4::psi2i(Interior)
  WRITES: WeylScal4::psi2r(Interior)
  WRITES: WeylScal4::psi1i(Interior)
  WRITES: WeylScal4::psi1r(Interior)
  WRITES: WeylScal4::psi0i(Interior)
  WRITES: WeylScal4::psi0r(Interior)
  WRITES: WeylScal4::NRPycurvIi(Interior)
  WRITES: WeylScal4::NRPycurvIr(Interior)
  WRITES: WeylScal4::NRPycurvJi(Interior)
  WRITES: WeylScal4::NRPycurvJr(Interior)
  WRITES: WeylScal4::NRPycurvJ1(Interior)
  WRITES: WeylScal4::NRPycurvJ2(Interior)
  WRITES: WeylScal4::NRPycurvJ3(Interior)
  WRITES: WeylScal4::NRPycurvJ4(Interior)

} "Call WeylScal4NRPy main function"


Overwriting WeylScal4NRPy/schedule.ccl


In [ ]:
#if (CCTK_EQUALS(output_all_psis,"true"))
#{
#    {
#  } /*"Allocate storage for the other psis if desired"*/
#    if (CCTK_EQUALS(output_invars,"true"))
#    {
#        {
#        } /*"Allocate storage for scalar invariants if desired"*/
#    }
#}

We will also need $\text{make.code.defn}$, which states which files need to be compiled. This thorn only has the one C file to compile.

In [19]:
%%writefile WeylScal4NRPy/src/make.code.defn

SRCS = WeylScal4NRPy.c

Overwriting WeylScal4NRPy/src/make.code.defn
